In [1]:
import pandas as pd
import numpy as np
import os
from dshortcuts.utils import open_pickle, save_pickle

In [2]:
# open prepared dtypes to save memory
trainDtypes = open_pickle(os.path.join('../input', 'trainDtypes.pickle'))
itemCategoriesDtypes = open_pickle(os.path.join('../input', 'itemCategoriesDtypes.pickle'))

In [3]:
train = pd.read_csv('../input/sales_train.csv', dtype=trainDtypes, parse_dates=['date'])
itemCategories = pd.read_csv('../input/item_categories.csv', dtype=itemCategoriesDtypes)
test = pd.read_csv('../input/test.csv', dtype=trainDtypes)
shops = pd.read_csv('../input/shops.csv')
items = pd.read_csv('../input/items.csv')

In [4]:
train.drop('item_price', axis=1, inplace=True)
# ?maybe we also don;t need the date since we have a block num

In [5]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype('int8')

In [6]:
train.dtypes, '------------------', test.dtypes

(date              datetime64[ns]
 date_block_num              int8
 shop_id                     int8
 item_id                    int16
 item_cnt_day             float16
 dtype: object,
 '------------------',
 ID                int64
 shop_id            int8
 item_id           int16
 date_block_num     int8
 dtype: object)

In [7]:
X = pd.concat([train.assign(dist='train'), test.assign(dist='test')], ignore_index=True, sort=False)

In [8]:
assert (len(train) + len(test)) == len(X)
X.shape, test.shape, train.shape

((3150049, 7), (214200, 4), (2935849, 5))

In [9]:
pd.merge?

In [12]:
_groupByCols = ['date_block_num', 'shop_id', 'item_id']
curMonSales = X.groupby(_groupByCols).item_cnt_day.sum().reset_index()
curMonSales['date_block_num'] = curMonSales['date_block_num'] + 1 
X = X.merge(curMonSales.rename({'item_cnt_day': 'prev_mon_sales'}, axis=1), how='left', on=_groupByCols)

In [32]:
_na_mask = X.prev_mon_sales.isna()
_probMissItems = X.loc[_na_mask].item_id.unique()
(~train.item_id.isin(_probMissItems)).sum()

0

In [43]:
X.prev_mon_sales = X.prev_mon_sales.fillna(0)
X.ID = X.ID.fillna(-1).astype('int32')

In [63]:
predictions = X[X.dist == 'test'][['ID', 'prev_mon_sales']]

In [64]:
predictions.prev_mon_sales = predictions.prev_mon_sales.clip(0, 20)
predictions = predictions.rename({'prev_mon_sales': 'item_cnt_month'}, axis=1)
predictions.describe()

,ID,item_cnt_month
count,214200.000000,214200.0
mean,107099.500000,0.0
std,61834.358168,0.0
min,0.000000,0.0
25%,53549.750000,0.0
50%,107099.500000,0.0
75%,160649.250000,0.0
max,214199.000000,20.0


In [65]:
predictions.to_csv('../output/T-1_benchmark.csv', index=False)